In [ ]:
%%bash
git clone https://github.com/ZhuoqunWang0120/cs671kaggle.git

Cloning into 'cs671kaggle'...


In [102]:
import lightgbm
import os
import pandas as pd
from sklearn import preprocessing
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score


In [ ]:
os.chdir('./cs671kaggle')

In [13]:
train_data = pd.read_csv('data/train.csv')
train_data = train_data.dropna()
# drop id
train_data = train_data.drop(columns = 'id')
# transform age to age groups
train_data.Age = list(map(lambda s: s[0], train_data.Age.astype('str')))
# transform dtype of temperature
train_data.Temperature = train_data.Temperature.astype('float')
# one-hot encoding
train_data = pd.get_dummies(train_data,dummy_na=False)

In [20]:
Xtrain, ytrain = train_data.drop(columns = 'Decision'), train_data.Decision

In [72]:
params ={'num_leaves': np.arange(5, 50, 1), 
             'min_data_in_leaf': np.arange(10, 200, 10)
}

In [85]:
clf = lightgbm.LGBMClassifier(max_depth = -1, random_state=0, silent=True, metric='None', n_estimators = 500)


In [86]:
lgb_cv = RandomizedSearchCV(
    estimator = clf, 
    n_iter = 100,
    param_distributions = params,
    scoring='accuracy',
    cv = 5, # 5fold
    random_state = 0,
    verbose = True)

In [88]:
lgb_cv.fit(Xtrain, ytrain)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: ignored

In [101]:
print(lgb_cv.best_params_, lgb_cv.best_score_)

{'num_leaves': 36, 'min_data_in_leaf': 70} 0.769532481633797


In [90]:
%%time
lgb_best = gs.best_estimator_.fit(Xtrain, ytrain)

CPU times: user 476 ms, sys: 12 ms, total: 488 ms
Wall time: 256 ms


In [106]:
np.mean(cross_val_score(lgb_best, Xtrain, ytrain, cv = 5, scoring = 'accuracy'))

0.7546704685732675

In [104]:
?cross_val_score

In [94]:
test_data = pd.read_csv('data/test.csv')
test_data.Age = list(map(lambda s: s[0], test_data.Age.astype('str')))
test_data.Temperature = test_data.Temperature.astype('float')
Xtest = test_data.drop(columns = 'id')
Xtest = pd.get_dummies(Xtest,dummy_na = False)

In [96]:
sub8 = pd.DataFrame({'id': test_data.id, 'Decision': lgb_best.predict(Xtest)})
sub8.to_csv('sub8.csv',index=False, header='sub8')

In [97]:
np.arange(5, 50, 1)

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [98]:
np.arange(10, 200, 10)

array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190])